In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import json
import re
import string
import nltk

In [23]:
df = pd.read_csv("preprocessed.csv", index_col=0)
df = df.reset_index()
df

,index,accessionNumber,filingDate,reportingDate,financialEntity,htmlFile,coDescription,CIK,name,countryinc,cityma,SIC,SIC_desc,coDescription_lemmatized,coDescription_stopwords
0,0,0001441816-18-000028,2018-03-30 20:12:23 UTC,2018-02-01,financialEntities/params;cik=1441816,https://www.sec.gov/Archives/edgar/data/144181...,"mongodb is the leading modern, general purpose...",1441816,"MONGODB, INC.",US,NEW YORK,7372,Prepackaged Software (mass reproduction of sof...,"mongodb is the leading modern , general purpos...",mongodb leading modern general purpose databas...
1,1,0001108524-18-000011,2018-03-09 22:01:46 UTC,2018-02-01,financialEntities/params;cik=1108524,https://www.sec.gov/Archives/edgar/data/110852...,salesforce is a global leader in customer rela...,1108524,SALESFORCE COM INC,US,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...,salesforce is a global leader in customer rela...,salesforce global leader customer relationship...
2,3,0001353283-18-000004,2018-03-30 21:21:46 UTC,2018-02-01,financialEntities/params;cik=1353283,https://www.sec.gov/Archives/edgar/data/135328...,splunk provides innovative software solutions ...,1353283,SPLUNK INC,NaN,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...,splunk provides innovative software solution t...,splunk provides innovative software solution e...
3,4,0001660134-18-000007,2018-03-12 20:45:43 UTC,2018-02-01,financialEntities/params;cik=1660134,https://www.sec.gov/Archives/edgar/data/166013...,okta is the leading independent provider of id...,1660134,"OKTA, INC.",US,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...,okta is the leading independent provider of id...,okta leading independent provider identity ent...
4,5,0001564590-18-007164,2018-03-29 21:34:05 UTC,2018-02-01,financialEntities/params;cik=1393052,https://www.sec.gov/Archives/edgar/data/139305...,veeva is a leading provider of industry cloud ...,1393052,VEEVA SYSTEMS INC,NaN,PLEASANTON,7372,Prepackaged Software (mass reproduction of sof...,veeva is a leading provider of industry cloud ...,veeva leading provider industry cloud solution...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,1114,0001595527-19-000005,2019-03-15 12:45:38 UTC,2019-01-01,financialEntities/params;cik=1595527,https://www.sec.gov/Archives/edgar/data/159552...,"ationwe were incorporated on december 19, 2013...",1595527,"AMERICAN REALTY CAPITAL NEW YORK CITY REIT, INC.",US,NEW YORK,6798,Real Estate Investment Trusts,"ationwe were incorporated on december 19 , 201...",ationwe incorporated december maryland corpora...
671,1115,0001144204-19-016652,2019-03-28 20:28:30 UTC,2019-01-01,financialEntities/params;cik=1130166,https://www.sec.gov/Archives/edgar/data/113016...,llowing business section contains forward-look...,1130166,"CYCLACEL PHARMACEUTICALS, INC.",US,BERKELEY HEIGHTS,2834,Pharmaceutical Preparations,llowing business section contains forward-look...,llowing business section contains statement ac...
672,1117,0001555280-19-000041,2019-02-14 22:08:33 UTC,2019-01-01,financialEntities/params;cik=1555280,https://www.sec.gov/Archives/edgar/data/155528...,ts.costs and expenses costs of sales consist p...,1555280,ZOETIS INC.,NaN,PARSIPPANY,2834,Pharmaceutical Preparations,ts.costs and expense cost of sale consist prim...,expense cost sale consist primarily cost mater...
673,1121,0001479094-19-000006,2019-02-13 21:22:54 UTC,2019-01-01,financialEntities/params;cik=1479094,https://www.sec.gov/Archives/edgar/data/147909...,certain definitionsin this report:we define ga...,1479094,"STAG INDUSTRIAL, INC.",NaN,BOSTON,6798,Real Estate Investment Trusts,certain definitionsin this report : we define ...,certain definitionsin report define gaap gener...


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

doc = CountVectorizer(ngram_range=(1,4))
word_count=doc.fit_transform(df["coDescription_stopwords"])

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=doc.get_feature_names(),columns=["idf_weights"])
df_idf.sort_values(by=['idf_weights']).head(40)

tf_idf_vector=tfidf_transformer.transform(word_count)
feature_names = doc.get_feature_names()

C:\Users\maryx\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\maryx\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
def getImportantWords(company_index):
    first_document_vector=tf_idf_vector[company_index]
    temp = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
    return temp.sort_values(by=["tfidf"],ascending=False).head(45)

In [25]:
## Get index of APPIAN, MONGODB, SALESFORCE
## APPIAN AND SALESFORCE HAVE SIMILAR RETURNS IN 2018
## APPIAN AND MONGODB HAVE DISSIMILAR RETURNS IN 2018

def getIndex(companyName):
    condition = df["name"] == companyName
    index = df.index
    return index[condition].tolist()

MONGODB = getIndex("MONGODB, INC.")[0]
APPIAN = getIndex("APPIAN CORP")[0]
SALESFORCE = getIndex("SALESFORCE COM INC")[0]
print(MONGODB)
print(APPIAN)

0
460


In [26]:
getImportantWords(MONGODB)

,tfidf
mongodb,0.389086
database,0.196785
developer,0.155608
platform,0.135699
mongodb atlas,0.128380
community server,0.121248
customer,0.118797
atlas,0.117314
server,0.109231
cloud,0.098725


In [27]:
getImportantWords(APPIAN)

,tfidf
platform,0.202909
appian,0.181988
customer,0.133505
software,0.113454
application,0.102286
sail,0.096750
organization,0.084140
user,0.083244
kx,0.079125
interface,0.071054


In [75]:
getImportantWords(SALESFORCE)

,tfidf
cloud,0.220839
customer,0.180660
apps,0.135817
salesforce,0.128698
service,0.119417
subscription,0.095385
salesforce platform,0.091341
platform,0.088367
solution,0.071163
service offering,0.069053


In [13]:
def findDuplicates(a, b):
    d = {}
    dup = []
    for word in a:
        d[word] = 1
    for word in b:
        if word in d:
            dup.append(word)
    return dup

def findDifferences(a, b):
    d = {}
    dup = []
    for word in a:
        d[word] = 1
    for word in b:
        if word not in d:
            dup.append(word)
    return dup

In [14]:
### Similarity between APPIAN and SALESFORCE
print(findDuplicates(getImportantWords(SALESFORCE).index.values, getImportantWords(APPIAN).index.values))
print(findDifferences(getImportantWords(SALESFORCE).index.values, getImportantWords(APPIAN).index.values))

IndexError: row index (747) out of range

In [81]:
### Similarity between APPIAN and MONGODB
findDuplicates(getImportantWords(MONGODB).index.values, getImportantWords(APPIAN).index.values)

['platform',
 'customer',
 'software',
 'application',
 'organization',
 'user',
 'subscription',
 'cloud',
 'enterprise',
 'deployment']

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
